<a href="https://colab.research.google.com/github/anelglvz/Working-Analyst/blob/main/ML-AI-for-the-Working-Analyst/Semana6_2_Working_Analyst_SistemadeRecomendacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install surprise

Consultar para mas detalles:
https://surprise.readthedocs.io/en/stable/index.html

### Introducción

En este ejemplo utilizaremos los datos de la [competencia de Netflix](https://www.kaggle.com/datasets/netflix-inc/netflix-prize-data?select=combined_data_1.txt) en Kaggle. El objetivo de esta competencia era mejorar el algoritmo de recomendación 10%. Nosotros no seremos tan avariciosos. Por lo que los objetivos de esta clase son: 



*   Análisis exploratorio de matriz y por qué es dispersa.
*   Implementación de Singular Value Decomposition.
*   Implementación de un modelo de sistema de recomendación de filtro colaborativo.
*   Generar la predicción de recomendaciones con buen resultado y no.

In [ ]:
import math 

import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.sparse import csr_matrix
from surprise import Reader, Dataset, SVD   # Simple Python RecommendatIon System Engine
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split


En este caso cargar los datos nos llevará un rato por la gran cantidad de registros que tenemos. Sólo cargaremos un archivo, pero la competencia tiene un total de 3 archivos.

In [ ]:
# Recuerde, pueden conseguir la dirección del archivo en su Drive y copiarla en la dirección
df = pd.read_csv('/content/drive/MyDrive/Curso-WorkingAnalyst/semana5/combined_data_1.txt', 
                 names=['Client_Id', 'Rating', 'Date'], low_memory=True, nrows=12*(10**6)) 
#df2 = pd.read_csv('/content/drive/MyDrive/Curso-WorkingAnalyst/semana5/combined_data_2.txt', 
#                 names=['Client_Id', 'Rating', 'Date'], low_memory=True) 
#df3 = pd.read_csv('/content/drive/MyDrive/Curso-WorkingAnalyst/semana5/combined_data_3.txt', 
#                 names=['Client_Id', 'Rating', 'Date'], low_memory=True) 
#df4 = pd.read_csv('/content/drive/MyDrive/Curso-WorkingAnalyst/semana5/combined_data_4.txt', 
#                 names=['Client_Id', 'Rating', 'Date'], low_memory=True) 
#                 nrows=12*(10**6))

En este dataset tenemos sólo tres columnas 

In [ ]:
# df1.shape, df2.shape, df3.shape, df4.shape

In [ ]:
# df = pd.concat([df1, df2, df3, df4])

In [ ]:
df

In [ ]:
df.info()

In [ ]:
# Revisamos los nulos. 

df.isnull().sum()

En este caso los registros que tenemos con valores nulos corresponden al Id de las películas. Si observamos los registros nulos están ordenados del 1 al 2,340.

In [ ]:
df[df['Rating'].isnull()].shape

In [ ]:
df[df['Rating'].isnull()].head()

In [ ]:
movies = df['Rating'].isnull().sum()
print(f'Este es el número de películas que tenemos en este archivo: {movies}')

In [ ]:
reviews = df[df['Rating'].isnull()==False]['Client_Id'].count()
print(f'Este es el número de calificaciones: {reviews}')

In [ ]:
users = df['Client_Id'].nunique() - movies
print(f'Esta es la cantidad de usuarios que tenemos: {users}')

En este caso no estaremos trabajando con las fechas. Por lo tanto haremos un subconjunto de nuestro DF original. 

In [ ]:
df_sub = df[['Client_Id', 'Rating']]

Ahora observemos como se distribuye la frecuencia para la columna de rating.

In [ ]:
sns.countplot(y=df_sub['Rating'], orient='v', palette='Blues');

In [ ]:
df_sub['Rating'].value_counts()/ df_sub['Rating'].count() * 100 

La mayor parte de nuestros valores se distribuyen en los ratings de 3 y 4 estrellas. La mayoría de los rating son positivos.

### Hora de la limpieza

In [ ]:
## Diferencia entre "Series" y "DataFrame"
df_aux = df.iloc[:5,:]
df_aux

In [ ]:
## Continuacion del anterior
# tipo DataFrame
df_aux[['Client_Id']]

In [ ]:
# tipo Series
df_aux['Client_Id']

En este caso los valores nulos que tenemos hacen referencia a **Id** de 'clientes' con rating vacíos. En realidad esta información es el **Id** de la película. Lo que debemos hacer ahora es quitar estos registros y añadir los **Id's** de película como una nueva columna. 

In [ ]:
# Generamos una serie con valores booleanos. Donde Verdadero será igual al lugar
# donde hay un Id de película.
pd.isnull(df_sub['Rating'][:100])

In [ ]:
# Colocamos la serie como un DataFrame
df_null = pd.DataFrame(pd.isnull(df_sub['Rating']))
df_null.head()

In [ ]:
# Obtenemos sólo los registros de películas junto con índice hasta donde llega 
# los rating para esa película.
df_null = df_null[df_null['Rating'] == True]
df_null.head()

In [ ]:
# Colocamos el índice como columna para tener la ubicación para hasta donde repe-
# tir nuestros valores de Id para esa película.
df_null = df_null.reset_index()
df_null

In [ ]:
movie_id_array = [] # Generamos una lista vacía donde colocaremos el Id de la película las veces que se repita.
movie_id = 1 # Inicializamos un contador

# En esta celda para saber cómo hace el proceso imprimo los distintos pasos.
for i, j in zip(df_null['index'][1:], df_null['index'][:-1]): # Iteramos sobre los valores de la columna 'index'
                                                               # empezando por el valor n+1 y en segundo lugar desde n hasta el penúltimo valor de la serie.
  temporary = np.full((1, i-j-1), fill_value=movie_id) # Creamos una matriz llena de valores con la forma de 1x(la diferencia del valor (n+1)-n-1).
                                                       # Esto nos da una matriz llena con el Id repetido el número de reviews para esa película.
  movie_id_array = np.append(movie_id_array, temporary) # Lo añadimos a una lista. 
  movie_id += 1                                         # Aumentamos el Id para la siguiente película.

In [ ]:
movie_id

In [ ]:
# Generamos los Id's para la última película que no está contemplada en nuestro loop.
last_movie = np.full((1, len(df_sub) - df_null.iloc[-1, 0] - 1), movie_id)
movie_id_array = np.append(movie_id_array, last_movie)

In [ ]:
movie_id_array.shape # la cantidad de Id's corresponde con la cantidad de películas.

In [ ]:
movie_id_array

Ahora para tener todo en orden es necesario que quitemos los registros nulos de nuestro df y añadamos los Id's que generamos.

In [ ]:
df_clean = df_sub[pd.notnull(df_sub['Rating'])].copy()

In [ ]:
df_clean['Movie_Id'] = movie_id_array.astype('int16') 

In [ ]:
df_clean['Client_Id'] = df_clean['Client_Id'].astype('int32')

In [ ]:
df_clean.head()

In [ ]:
df_clean.tail()

In [ ]:
df_clean.info()

In [ ]:
df_clean.isnull().sum().sum()



---



---



### Removiendo datos con poco valor informativo 

Nuestros datos se encuentran en el formato correcto. Sin embargo, existen muchos datos que no ayudan a nuestra predicción. ¿Por qué sucede esto? En esto momento no parece que tenemos valores nulos, sin embargo no todos los usuarios han calificado las 4,499 películas que tenemos. De aquí va a provenir nuestra matriz dispersa. 

Pero antes de pasar a la matriz intentemos reducir la cantidad de datos que tenemos removiendo dos casos de nuestro dataset:
1. Las películas que tiene pocas calificaciones. 
2. Usuarios que han calificado muy pocas películas.

Las películas que tiene pocas calificaciones.

In [ ]:
# Agregamos a nivel película para revisar el conteo y valor promedio para cada una.
movie_agg = df_clean.groupby('Movie_Id').agg({'Rating': ['count', 'mean']})
movie_agg

In [ ]:
movie_agg.info()

In [ ]:
movie_agg.describe()

Observamos el comportamiento de la distribución del conteo.

In [ ]:
sns.boxplot(x=movie_agg[('Rating', 'count')]);

In [ ]:
sns.displot(data=movie_agg, x=('Rating', 'count'), log_scale=True);

Por el comportamiento de la distribución del conteo de nuestros ratings observamos que hay un número considerable de películas con un muy pocas calificaciones. Revisemos cuántas son.

In [ ]:
# Tomamos el valor del tercer cuartil.
movies_low_rating = movie_agg[('Rating', 'count')].quantile(0.75)
movies_low_rating

In [ ]:
# Seleccionamos todas las películas que están por debajo del valor del cuartil
movies_to_drop = movie_agg[movie_agg[('Rating', 'count')] < movies_low_rating].index
movies_to_drop.shape

In [ ]:
movies_to_drop

Ahora observemos el caso del conteo para los usuarios. Usuarios que han calificado muy pocas películas.

In [ ]:
user_agg = df_clean.groupby('Client_Id').agg({'Rating': ['count', 'mean']})['Rating']
user_agg.head()

In [ ]:
user_agg.info()

In [ ]:
user_agg.describe()

In [ ]:
sns.boxplot(x=user_agg['count']);

In [ ]:
sns.displot(data=user_agg, x='count', log_scale=True);

In [ ]:
user_low_rating = user_agg['count'].quantile(0.75)
user_low_rating

In [ ]:
users_to_drop = user_agg[user_agg['count'] < user_low_rating].index
users_to_drop.shape

In [ ]:
users_to_drop

Removamos las películas con bajo número de calificaciones y los usuarios con pocas películas calificadas.

In [ ]:
df_clean

In [ ]:
df_trim = df_clean[~df_clean['Movie_Id'].isin(movies_to_drop)]

In [ ]:
df_clean.shape, df_trim.shape

In [ ]:
df_trim = df_trim[~df_trim['Client_Id'].isin(users_to_drop)]

In [ ]:
df_clean.shape, df_trim.shape

In [ ]:
df_trim.shape[0] / df_clean.shape[0]

Eliminamos casi el 40% de nuestros datos. 

### Convirtiendo a matriz dispersa o generando nuestra tabla Usuario-Item

In [ ]:
df_trim

In [ ]:
# Por limitantes de la ramm, no podemos crearlo, pero en local o usando otras herramientas, podrían 
%%time
pivot_user_item = df_trim.pivot(index='Client_Id', columns='Movie_Id', values='Rating').fillna(0)

In [ ]:
pivot_user_item.head()

#### Cargando dataset con información de películas.

Hay que utilizar el mismo nombre para la columna del Id de película, esto nos servirá después para cruzar con nuestra recomendación.

In [ ]:
df_movies = pd.read_csv('/content/drive/MyDrive/Curso-WorkingAnalyst/semana5/movie_titles.csv', 
                        encoding='ISO-8859-1', names=['Movie_Id', 'Year', 'Title'])

df_movies = df_movies[df_movies['Movie_Id'] < 2341]
df_movies

In [ ]:
df_movies.set_index('Movie_Id', inplace=True) 

In [ ]:
df_movies.head()

### Haciendo una recomendación

In [ ]:
# Obtengamos un usuario al azar
np.random.seed(2)
random_user_id = np.random.choice(df_trim['Client_Id'])
random_user_id

In [ ]:
# Veamos qué le ha gustado antes a este usuario o ha visto

user = df_trim[(df_trim['Client_Id'] == random_user_id)].set_index('Movie_Id')
user_rated_movies = user.join(df_movies)
user_rated_movies 

In [ ]:
user_rated_movies['Rating'].value_counts()

Recordemos que eliminamos de nuestra matriz películas que no tenían una cantidad considerable de calificaciones, así que debemos eliminarlas también de este listado de películas que utilizaremos para recomendar.

In [ ]:
movies_to_not_considered = movies_to_drop.to_list() + user_rated_movies.index.to_list()

In [ ]:
create_recom = df_movies.copy()
create_recom = create_recom.reset_index()
create_list_of_possible_movies_to_recommend = create_recom[(~create_recom['Movie_Id'].isin(movies_to_not_considered))] 

create_list_of_possible_movies_to_recommend

### Singular Value Decomposition (SVD)

La descomposición en valores singulares nos permite reducir las dimensiones de nuestra matriz a partir de su factorización. 

Ahora ya tenemos nuestra matriz factorizada y lista para generar recomendaciones. Pero antes de eso es necesario traer el nombre de nuestras películas. 

In [ ]:
# Obtenemos un Id de una película para estimar su posible valor de calificación.
np.random.seed(0)
random_movie_id = create_list_of_possible_movies_to_recommend.sample()
print(random_movie_id)
random_movie_id = random_movie_id.loc[:, 'Movie_Id'].values[0]
print(f'random_movie_id is {random_movie_id}')



---



---



In [ ]:
%%time
# Utilizamos las clases y funciones que importamos del módulo surprise (Reader, Dataset, SVD)
reader = Reader(rating_scale=(1, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df_trim[['Client_Id', 'Movie_Id', 'Rating']], reader)

trainset = data.build_full_trainset() # cargamos todos nuestros datos en un objeto tipo trainset


svd = SVD(biased=False) 
svd.fit(trainset) # Ajustamos nuestros datos a la matriz que factorizamos. 

In [ ]:
# Hacemos la predicción y obtenemos el valor estimado de la calificación.
prediction = svd.predict(random_user_id, random_movie_id)
print(prediction)

In [ ]:
user_recommendations = create_list_of_possible_movies_to_recommend.copy()
user_recommendations['Estimated_Rating'] = user_recommendations['Movie_Id'].apply(lambda x: svd.predict(random_user_id, x).est)

In [ ]:
user_recommendations.head()

In [ ]:
user_recommendations.sort_values('Estimated_Rating', ascending=False)



---



---



Hacemos lo anterior con datos train and test

In [ ]:
%%time
# Utilizamos las clases y funciones que importamos del módulo surprise (Reader, Dataset, SVD)
reader = Reader(rating_scale=(1, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df_trim[['Client_Id', 'Movie_Id', 'Rating']], reader)

# # test set is made of 25% of the ratings.
trainset, testset = train_test_split(data, test_size=0.25)

svd = SVD(biased=False) 
svd.fit(trainset) # Ajustamos nuestros datos a la matriz que factorizamos. 

In [ ]:
# Hacemos la predicción y obtenemos el valor estimado de la calificación.
prediction = svd.predict(random_user_id, random_movie_id)
prediction

In [ ]:
svd.test(testset)



---



---



Hacemos lo anterior con `cross_validate` y datos de test

In [ ]:
%%time
# Utilizamos las clases y funciones que importamos del módulo surprise (Reader, Dataset, SVD)
reader = Reader(rating_scale=(1, 5))


data = Dataset.load_from_df(df_trim[['Client_Id', 'Movie_Id', 'Rating']],
                            reader) # Es necesario pasar en este orden las columnas

svd = SVD(biased=False) 
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=3)